In [ ]:
### Temporaire
df.drop(['code_postal'],axis=1, inplace=True) # Dois utiliser target encoder
df.drop(['nombre_pieces_principales'],axis=1, inplace=True) # 
df.drop(['code_type_local'],axis=1, inplace=True)

In [ ]:
print('1er lot: {}'.format(len(df[df.surface_carrez_du_1er_lot!=0])))
print('2eme lot: {}'.format(len(df[df.surface_carrez_du_2eme_lot!=0])))
print('3eme lot: {}'.format(len(df[df.surface_carrez_du_3eme_lot!=0])))
print('4eme lot: {}'.format(len(df[df.surface_carrez_du_4eme_lot!=0])))
print('5eme lot: {}'.format(len(df[df.surface_carrez_du_5eme_lot!=0])))

surface = df[(df.surface_carrez_du_1er_lot!=0) & 
             (df.surface_carrez_du_2eme_lot!=0) & 
             (df.surface_carrez_du_3eme_lot!=0) & 
             (df.surface_carrez_du_4eme_lot!=0) &
             (df.surface_carrez_du_5eme_lot!=0)]

print('Tous les lots: {}'.format(len(surface)))

lot_et_prix = df[(df.surface_carrez_du_1er_lot!=0) & (df.valeur_fonciere!=0)]

print('1er lot avec prix: {}'.format(len(lot_et_prix)))

In [ ]:
## Sanity Check
missing_values_assessment(df)

## Start ML

#### Découpage des données pour l'entraînement et l'évaluation de l'apprentissage automatique

In [ ]:
from sklearn.model_selection import train_test_split

y = df['valeur_fonciere']
X = df.drop(['valeur_fonciere'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.33, random_state=42)

print(X_train.shape)
print(y_train.shape)

#### Régression linéaire

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train,y_train)

In [ ]:
y_pred = reg.predict(X_test)
reg.score(X_test, y_test)

#### Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
cv_reg = LinearRegression()
cv_results = cross_val_score(cv_reg, X, y, cv=100)
print(cv_results)

In [ ]:
np.mean(cv_results)

#### Régression linéaire avec Lasso

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.4, normalize=True)
lasso.fit(X_train,y_train)

#### Filtre sur l'importance des features

In [ ]:
lasso_df = pd.DataFrame(np.array([lasso.coef_]), columns=X.columns)

for col in lasso_df.columns:
    if lasso_df[col].iloc[0]<1000:
        lasso_df.drop(col, axis=1, inplace=True)

In [ ]:
col_rang = range(len(lasso_df.columns))

plt.figure(figsize=(20,10))
plt.bar(col_rang, lasso_df.iloc[0])
plt.xticks(col_rang, lasso_df.columns, rotation=60)
plt.margins(0.02)
plt.show()